In [13]:
import numpy as np
import pandas as pd
#from __future__ import division

np.random.seed(seed=42)

#demand = pd.DataFrame({ 'ID' : np.arange(200),
#                              'population' : np.random.randint(low=4000, high=10000, size=200),
#                              'area' : np.random.randint(low=1, high=50, size=200),
#                              'user_demand' : [50]*200})

population = np.random.randint(low=4000, high=10000, size=200)
area = np.random.randint(low=1, high=50, size=200)
user_demand = [50]*200
penetration = 0.8
OBF = 50


def calc_demand(population, area, user_demand, penetration):
    users = population*penetration
    density = users / area #km2
    user_throughput = user_demand
    capacity_required_km2 = user_throughput * density
    total_capacity_required_km2 = capacity_required_km2 / OBF
    print total_capacity_required_km2

 
    
    
    
    
    
calc_demand(population, area, user_demand, penetration)

#calculate number of users, here assuming 80% penetration 
#demand['users'] = np.round(demand.population*0.8)

#calculate user density by dividing users by geographical area
#demand['user_density'] = np.round(demand.users / demand.area, 1)

#calculate average user demand
#demand['ave_user_throughput'] = np.round(demand.user_demand*1024*8*(1/1)*(1/30)*(1/3600),1)

#calculate capacity per area unit (Mbps/km2)
#demand['capacity_required_km2'] = np.round(demand.ave_user_throughput * demand.user_density,1)

#calculate overbooking factor of 50:1
#demand['total_capacity_required_km2'] = np.round(demand.capacity_required_km2 / 50,1)




[  324.           192.61538462  3690.4         2450.93333333   126.88979592
   153.34054054   158.92244898   210.16470588   152.39183673  3370.4
  3831.2          212.68571429   249.42608696   149.6          103.25
   137.81818182  3815.2          269.09473684  3844.4          116.96363636
   286.49230769   226.275        691.2          213.875       1477.
   590.32727273   416.14117647   145.34736842   149.13333333   813.12
   154.30588235   806.26666667   245.34545455   361.81818182    96.76666667
   209.45         163.24848485   622.66666667  1154.4          294.95
   145.29655172  1474.13333333   288.08         167.91111111   332.21052632
   142.95384615  1836.8          395.49473684   257.4          115.           727.2
   118.26341463   102.81212121   120.82         139.97948718   293.86666667
   107.26        7588.8          398.4          187.31428571   152.8
   278.74782609   104.10322581   190.72         113.42857143   127.65714286
   907.77142857   353.65         154.4923076